In [4]:
import requests
import urllib.request
import pandas as pd
import warnings
import time
import json
import base64
import csv
import numpy as np
import os
import glob
warnings.filterwarnings("ignore")

In [5]:
# read all sequence_key from file sequnce_key 
def read_sequnce_key_from_file():
    sequence_key_list = []
    for file_name in glob.glob("seqns/*.txt"):
        with open(file_name) as f:
            lines = f.readlines()
            lines = [line.strip() for line in lines if line.strip()]
            sequence_key_list.extend(lines)
    return list(set(sequence_key_list))

In [6]:
def get_images_ids_from_sequence(app_access_token: str, sequence_id: str) -> dict:
    '''
    Get all image_ids from a specific driving trip using sequence_id
    INPUTS: 
        app_access_token: this is a string. We can create your access token at https://mapillary.com/developer and get it using the link "https://www.mapillary.com/dashboard/developers"
        sequence_id: this is string. We can get it from a website after a specific trip. We want to get all the images. Note: all images in one trip get the same sequence_id.
    OUTPUTS:
        sequence of images_ids ordered by capture time
    '''
    # Adapted from: https://help.mapillary.com/hc/en-us/articles/360010234680-Accessing-imagery-and-data-through-the-Mapillary-API
    url = 'https://graph.mapillary.com/image_ids?sequence_id={}&access_token={}'.format(sequence_id, app_access_token)
    headers = { "Authorization" : "OAuth {}".format(app_access_token) }
    response = requests.get(url, headers)
    data = response.json()
    return data

In [7]:
def get_image(app_access_token: str, image_id: str, num_img: int, image_size = 'thumb_2048_url')-> dict:
    '''
    get all data related to image
    INPUTS:
     app_access_token: app_access_token: this is a string. We can create your access token at https://mapillary.com/developer and get it using the link "https://www.mapillary.com/dashboard/developers"
     image_id: string we can get it from website or from function get_images_ids_from_sequence
     detection_classes: list of string containing detection classes needed to detect and set it empty list when need to download all images not depending on specific detection by defult empty list
     image_size: string specify number of pixels of image there are 4 options ["thumb_256_url", "thumb_1024_url", "thumb_2048_url", "thumb_original_url"]

    OUTPUTS:
     image_id: image id 
     captured_at: string timestamp epoch
     height: 2048 image height 
     width: 1536 image width
     detections: list with objects detections ids
     
     return:
      image downloaded or not
    '''
    # Adapted from: https://help.mapillary.com/hc/en-us/articles/360010234680-Accessing-imagery-and-data-through-the-Mapillary-API
    url = 'https://graph.mapillary.com/{}?fields=id,captured_at,height,width,detections.value,{}&access_token={}'.format(image_id, image_size, app_access_token)
    headers = { "Authorization" : "OAuth {}".format(app_access_token) }
    response = requests.get(url, headers)
    data = response.json()
    location = './imgs'
    image_name = str('{0:05}'.format(num_img))+'_'+data['id']+'.jpg'
    image_path = os.path.join(location, image_name)
    images_names.append(image_path)
    urllib.request.urlretrieve(data[image_size], image_path)
    print(image_name)

In [ ]:
sequence_key_list = read_sequnce_key_from_file()
rest_seq=read_sequnce_key_from_file()

In [ ]:
app_access_token = 'MLY|5301732853205994|611e88a33f1ef1f26c9347b9b43bbc68'
images_names = []
# Get every sequence id in the sequences list.
for sequence_id in sequence_key_list:
    print(sequence_id,'Started')
    # Send a request and download every image with the same sequece id
    for img_id in (get_images_ids_from_sequence(app_access_token, sequence_id)['data']):
        get_image(app_access_token, img_id['id'], len(images_names))
    rest_seq.remove(sequence_id)
    print(sequence_id,'Done')